In [ ]:
# Allows the modification of the files imported without having to restart kernel or re-import them
%load_ext autoreload
%autoreload 2

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython import display
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

# to have interactive plots : pip install ipympl
%matplotlib ipympl

### General imports
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pickle

plt.rc('figure',figsize=(10,6))
plt.rc('font',size=12)

### Astropy configuration
import astropy.units as u

#### QUBIC IMPORT
from qubic.lib import Qdictionary 
from qubic.lib.Instrument import Qinstrument
import qubic.lib.Calibration.Qfiber as ft
import qubicpack.qubicfp as qfp
from qubicpack.plot_fp import plot_fp
# from qubicpack.qubicfp import qubicfp

d = Qdictionary.qubicDict()
dictfilename = '/dicts/global_source_oneDet.dict'
d.read_from_file(dictfilename)
q = Qinstrument.QubicInstrument(d)

# Same for all maps
nside = 256
azqubic = 116.4

### Temporary update of the path 
in order to be able to load libraries that are still in development and not yet in the QUBIC path. This will have to be removed when the relevant libraries are finalized and integrated into QubicSoft

In [ ]:
dirtemplibs = ["/Users/huchet/qubic/qubic/scripts/MoonProject/", "/Users/huchet/Documents/code/scripts/", "/Users/huchet/Documents/code/data/"] #[os.environ['QUBIC_DATADIR']+'scripts/MoonProject/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
import fitting as fit
import time_domain_tools as tdt
import pipeline_moon_plotting as pmp
import pipeline_moon_functions as pmf


## Dataset and Observing site
in the case of the July 2022 Moon observations, they were from Salta CNEA Regional and the UTC offset was -3 hours.

### Location of the raw TOD files
Beware this will have to be changed for each one's configuration

In [ ]:
mydatadir = '/Users/huchet/qubic/qubic/data/Calib-TD/'

### Observation date and corresponding file

In [ ]:
ObsDate = '2025-05-22'
ObsSession = 0
dirs = glob.glob(mydatadir + ObsDate + '/*NoiseMeasurement*')
print(dirs)
datadir = dirs[0]


### Observing Site

In [ ]:
La_Puna = {'lat':-24.186971*u.deg,
            'lon':-66.478209*u.deg,
            'height':4826*u.m,
            'UTC_Offset':-3*u.hour}

Obs_Site = La_Puna

In [ ]:
# try:
#     data_TOD
# except NameError:
print('Reading data from disk: '+datadir)
a = qfp.qubicfp()
a.read_qubicstudio_dataset(datadir)
tt, alltod = a.tod()
az = a.azimuth()
el = a.elevation()
thk = a.timeaxis(datatype='hk')
tinit = tt[0]
# tt, alltod, thk, az, el, tinit = pmf.read_data(datadir, remove_t0=False)
# az += azqubic
# tt_save = np.copy(tt)
# alltod_save = np.copy(alltod)
# thk_save = np.copy(thk)
# az_save = np.copy(az)
# el_save = np.copy(el)
# data_TOD = [tt_save, alltod_save, thk_save, az_save, el_save, tinit]
# print("tinit = {}".format(tinit))

In [ ]:
a.plot_timeline(93,1) # plot the timeline for TES 93 of ASIC 1
a.plot_timeline(93,2) # plot the timeline for TES 93 of ASIC 2

In [ ]:
ASICNum = 1
TESNum = 66 + (ASICNum - 1)*128
tod = alltod[TESNum - 1, :]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(tt, tod, label="TOD")
# ax.plot(tt, mytod, label="filtered TOD")
ax.legend()
ax.set_xlabel("Time [s]")
ax.set_ylabel("Flux [ADU]")
plt.tight_layout()
plt.savefig("tod_filtering.pdf")
plt.show()